# Correction TP 2

In [6]:
import masterjdv2 as jdv
import numpy as np

## Creation d'une grille 2000 x 2000

In [7]:
grid = jdv.init_grid(2000)

## Fonction d'élargissement de la grille
Si la grille initiale est de taille idim X jdim alors la grille élargie est de taille idim+2 X jdim+2 

In [8]:
def enlarge_grid(grid):
    idim,jdim = grid.shape
    res_grid = np.zeros((idim + 2,jdim + 2))
    res_grid[1:idim + 1,1:jdim + 1] = grid[:,:]
    return res_grid

In [14]:
def evolution_eg(grid):
    # grid is an enlarge grid
    idim = grid.shape[0]
    jdim = grid.shape[1]
    res_grid = np.zeros((idim,jdim))
    # we start from 1 to idim-1
    for i in range(1,jdim - 1):
        for j in range(1,jdim - 1):
            # by default cell_state is dead
            cell_state = 0
            # loop over neighs to count living cells
            nb_neigh = 0
            # loop over neighs to count living cells
            for ii in [i - 1,i,i + 1]:
                for jj in [j - 1,j,j + 1]:
                    if grid[ii,jj] == 1:
                        nb_neigh = nb_neigh + 1
            # in my count I count current cell itself substract the value
            nb_neigh = nb_neigh - grid[i,j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh == 2:
                cell_state = grid[i,j]
            # if 3 neighbors cell state is alive
            elif nb_neigh == 3:
                cell_state = 1
            # in other case cell state is dead (default state)
            res_grid[i,j] = cell_state
    return res_grid

## Création d'une grille élargie

In [10]:
egrid = enlarge_grid(grid)

## Comparaison des temps d'exécution

In [29]:
%timeit jdv.evolution1(grid)

40.1 s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit jdv.evolution2(grid)

32.1 s ± 2.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit evolution_eg(egrid)

28 s ± 2.91 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Résultats gains de temps
rappel evolution2 : ~ 20% de par rapport à evolution1

evolution_eg : ~ 16% par rapport à evolution2 ; ~ 35% par rapport à evolution1

## Bonus : Unrolling
Dans evolution_eg on réalise deux boucles imbriquées de i-1 à i+1 et de j-1 à j+1 on peut écrire explicitement la somme des termes, il s'agit d'un procédé d'unrolling (déroulement des petites boucles), on peut même supprimer le terme à la position i,j

In [34]:
def evolution_unroll(grid):
    idim = grid.shape[0]
    jdim = grid.shape[1]
    res_grid = np.zeros((idim,jdim))
    for i in range(1,idim - 1):
        for j in range(1,jdim - 1):
            nb_neigh = grid[i - 1,j - 1] + grid[i - 1,j] + grid[i - 1,j + 1] \
                     + grid[i,j - 1]     + grid[ i,j + 1] \
                     + grid[i + 1,j - 1] + grid[i + 1,j] + grid[i + 1,j + 1]  
            if nb_neigh == 2:
                res_grid[i,j] = grid[i,j]
            elif nb_neigh == 3:
                res_grid[i,j] = 1
    return res_grid

In [35]:
%timeit evolution_unroll(egrid)

15.2 s ± 1.38 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Résultats gains de temps
evolution_unroll : 46% de par rapport à evolution_eg ; 65% par rapport à evolution1

## Store
Pour le stockage on peut également utiliser de l'unrolling, par contre on ne doit pas supprimer le terme i,j car cela créerait un biais

In [38]:
def evolution_store(grid):
    idim,jdim= grid.shape
    res_grid = np.zeros((idim,jdim))
    for i in range(1,idim-1):
        #specific case j=1 for initial storage values
        store0 = 0 # because store0=grid[i-1,0]+grid[i,0]+grid[i+1,0] all zeros
        store1 = grid[i - 1,1] + grid[i,1] + grid[i + 1,1] # j=1
        store2 = grid[i - 1,2] + grid[i,2] + grid[i + 1,2] # j+1=2
        nb_neigh = store0 + store1 + store2 - grid[i,1] # substract the element (i,j)
        if nb_neigh == 2:
            res_grid[i,1] = grid[i,1]
        elif nb_neigh == 3:
            res_grid[i,1] = 1
        # loop on j (we begin with j=2)
        for j in range(2,jdim - 1):
            # switch the storage values
            store0 = store1
            store1 = store2
            #compute the 3rd storage value
            store2 = grid[i - 1,j + 1] + grid[i,j + 1] + grid[i + 1,j + 1]   # 3rd subcolumn
            # compute nb_neigh
            nb_neigh = store0 + store1 + store2 - grid[i,j] # substract the element (i,j)
            if nb_neigh == 2:
                res_grid[i,j] = grid[i,j]
            elif nb_neigh == 3:
                res_grid[i,j] = 1
    return res_grid

In [39]:
%timeit evolution_store(egrid)

8.31 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Résultats gains de temps
evolution_store : ~ 46% de par rapport à evolution_unroll ; ~ 80% par rapport à evolution1